Type checking via a decorator.

In [1]:
import pandas as pd
from type_signature import TypeSignature

We add a decorator that shows the types of at least a subset of positional and named arguments. Declarations are either Python types, or sets of types. A special case is Pandas data frames, where we specify a required subset of columns and their value type-sets. "return_spec" is reserved to name the return type of the function (so the function we are working with may not have an argument of that name).

In [2]:
@TypeSignature({
        'a': int, 
        'b': int, 
        'c': pd.DataFrame({'x': [int]}),
        },
        return_spec=pd.DataFrame({'z': [float]}))
def fn(a, /, b, *, c, d=None):
    """doc"""
    return d

In [3]:
help(fn)

Help on function fn in module __main__:

fn(a, /, b, *, c, d=None)
     arg specifications
    {'a': {<class 'int'>},
     'b': {<class 'int'>},
     'c':                  x
    0  {<class 'int'>}}
     return specification:
                       z
    0  {<class 'float'>}
    
    
    doc



In [4]:
try:
    fn()
except TypeError as e:
    print(e)

fn() expected arg a
fn() expected arg b
fn() expected arg c


In [5]:
try:
    fn(1, 2)
except TypeError as e:
    print(e)

fn() expected arg c


In [6]:
try:
    fn(1, 2, c=3)
except TypeError as e:
    print(e)

fn() arg c should be type pandas.DataFrame, was type int


In [7]:
try:
    fn(1, 2, c=pd.DataFrame({'z': [7]}))
except TypeError as e:
    print(e)

fn() arg c missing required column 'x'


In [8]:
fn(1, 2, c=pd.DataFrame({'x': [3]}), d=pd.DataFrame({'z': [7.0]}))

,z
0,7.0


In [9]:
fn(1, b=2, c=pd.DataFrame({'x': [3]}), d=pd.DataFrame({'z': [7.0]}))

,z
0,7.0


In [10]:
try:
    fn(1, 2, c=pd.DataFrame({'x': [3.0]}))
except TypeError as e:
    print(e)

fn() arg c  column 'x' has a type float entry, when one of {int} expected


In [11]:
try:
    fn(1, 2, c=pd.DataFrame({'x': [30], "z": [17.2]}), d=pd.DataFrame({'q': [7.0]}))
except TypeError as e:
    print(e)

fn() return value: missing required column 'z'


In [12]:
@TypeSignature(
        {'a': pd.DataFrame},
        return_spec=int,
)
def g(a):
    return a.shape[0]

In [13]:
try:
    g(a=7)
except TypeError as e:
    print(e)

g() arg a one of {DataFrame} expected, was type int


In [14]:
try:
    g(7)
except TypeError as e:
    print(e)

g() arg a one of {DataFrame} expected, was type int


In [15]:
g(a=pd.DataFrame({'x': [5]}))

1

In [16]:
try:
    g({'x': [5]})
except TypeError as e:
    print(e)

g() arg a one of {DataFrame} expected, was type dict
